In [25]:
# Imports
import numpy as np
import spikeinterface as si

In [26]:
# Properties of the in-memory dataset
num_channels=7
samplerate=30000
duration=20
num_timepoints=int(samplerate*duration)
num_units=4
num_events=1000

In [27]:
# Generate a pure-noise timeseries dataset and a linear geometry
timeseries=np.random.normal(0,10,(num_channels,num_timepoints))
geom=np.zeros((num_channels,2))
geom[:,0]=range(num_channels)

# Define the in-memory input extractor
IX=si.NumpyInputExtractor(timeseries=timeseries,geom=geom,samplerate=samplerate)

# Generate some random events
times=np.sort(np.random.uniform(0,num_timepoints,num_events))
labels=np.random.randint(1,num_units+1,size=num_events)
    
# Define the in-memory output extractor
OX=si.NumpyOutputExtractor()
for k in range(1,num_units+1):
    times_k=times[np.where(labels==k)[0]]
    OX.addUnit(unit_id=k,times=times_k)

In [28]:
# Demonstrate the API for extracting information
print('Unit ids = {}'.format(OX.getUnitIds()))
st=OX.getUnitSpikeTrain(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=OX.getUnitSpikeTrain(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1, 2, 3, 4]
Num. events for unit 1 = 234
Num. events for first second of unit 1 = 16


In [29]:
# Write the input/output in the MountainSort format
si.MdaInputExtractor.writeDataset(input_extractor=IX,output_dirname='sample_mountainsort_dataset')
si.MdaOutputExtractor.writeFirings(output_extractor=OX,firings_out='sample_mountainsort_dataset/firings_true.mda')

In [30]:
# Read this dataset with the Mda input extractor
IX2=si.MdaInputExtractor(dataset_directory='sample_mountainsort_dataset')
OX2=si.MdaOutputExtractor(firings_file='sample_mountainsort_dataset/firings_true.mda')

In [31]:
# We should get he same information as above
print('Unit ids = {}'.format(OX.getUnitIds()))
st=OX2.getUnitSpikeTrain(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=OX2.getUnitSpikeTrain(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1, 2, 3, 4]
Num. events for unit 1 = 234
Num. events for first second of unit 1 = 16


In [32]:
# For development purposes, reload imported modules when source changes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# Note that the following only works if you point the below path to the spike-collab directory

import sys, os
sys.path.append(os.getcwd()+'/../../spike-collab')
from widgets import TimeseriesWidget
from widgets import UnitWaveformsWidget

In [ ]:
TimeseriesWidget(input_extractor=IX).plot()

In [ ]:
UnitWaveformsWidget(input_extractor=IX2,output_extractor=OX2).plot()